#### KoAlpaca-Polyglat-12.8B f16 safetensors 모델 
1. 모델 정보
   * 파일형식: safetensors
   * f16 모델

2. 학습훈련 파라메터
   * learning_rate: 5e-05
   * train_batch_size: 1
   * seed: 42
   * distributed_type: multi-GPU (A100 80G)
   * num_devices: 4
   * gradient_accumulation_steps: 64
   * total_train_batch_size: 256
   * total_eval_batch_size: 32
   * optimizer: Adam with betas=(0.9,0.999) and epsilon=1e-08
   * lr_scheduler_type: linear
   * num_epochs: 2.0

3. Framework versions
   * Transformers 4.28.1
   * Pytorch 2.0.0+cu117
   * Datasets 2.11.0
   * Tokenizers 0.13.3

4. requirement.txt
   * torch
   * transformers
   * accelerate
   * bitsandbytes

In [ ]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

In [ ]:

model_id = "/data/bwllm/models/koalpaca-polyglot-12.8bsf"
#model_id = "/data/bwllm/models/casllm-base-7b-sf"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto", 
    revision="8bit",
    load_in_8bit=True,
    # 아래의 max_memory 코드는 문제가 있는 것으로 보임 에러가 발생함.
    # max_memory=f'{int(torch.cuda.mem_get_info()[0]/1024**3)-2}GB'
)

In [ ]:
# 모델의 첫 번째 파라미터의 데이터 타입 확인
first_param_dtype = next(model.parameters()).dtype
print(f"First parameter data type: {first_param_dtype}")

In [ ]:
# 양자화 여부 확인
if 'quantized' in str(first_param_dtype):
    print("The model is quantized.")
else:
    print("The model is not quantized.")

In [ ]:
print(model)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

# 장치 확인   
print(device)

# 모델을 해당 디바이스로 이동
model.to(device)

In [ ]:
# 모델의 모든 파라미터의 데이터 타입 확인
for name, param in model.named_parameters():
    print(f"{name} data type: {param.dtype}")

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=model_id,
    #device=0,
)

In [ ]:
def make_answer(prompt_status, question):
    
    messages = prompt_status + [ {"role": "질문", "content": question}]    
    prompt_text = "\n".join(
        [f"### {msg['role']}:\n{msg['content']}" for msg in messages]
    )
    return prompt_text

In [ ]:
# 문장 생성
def generate(prompt_text):
    answer = pipe(
        prompt_text,
        do_sample=True,
        max_new_tokens=2000,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False,
        eos_token_id=2,
    )
    return answer[0]['generated_text']

In [ ]:
prompt_status = [
            {
                "role": "맥락",
                "content": "KoAlpaca(코알파카)는 EleutherAI에서 개발한 Polyglot-ko 라는 한국어 모델을 기반으로, 자연어 처리 연구자 Beomi가 개발한 모델입니다.",
            },
            {
                "role": "맥락",
                "content": "ChatKoAlpaca(챗코알파카)는 KoAlpaca를 채팅형으로 만든 것입니다.",
            },
            {"role": "명령어", "content": "친절한 AI 챗봇인 ChatKoAlpaca 로서 답변을 합니다."},
            {
                "role": "명령어",
                "content": "인사에는 짧고 간단한 친절한 인사로 답하고, 아래 대화에 간단하고 짧게 답해주세요.",
            },
        ]

prompt_status2 = []

question = "정보화시스템에서의 감리절차에 대해 알려줘"
prompt_text = make_answer(prompt_status2, question)

print(prompt_text)

In [ ]:
# 답변 텍스트만 출력
answer = generate(prompt_text)
print(answer)